In [25]:
import os,sys
import pandas as pd
sys.path.append(os.path.abspath('./../../'))
from Controllers.Controller import Controller
from Controllers.CellTypeController import CellTypeController
from model.annotation_points import AnnotationSession
from Controllers.MarkedCellController import MarkedCellController
from cell_extractor.diagnostics.ToolKit import get_DataFrame_from_query_result
from model.cell_type import CellType


In [26]:
controller = Controller()
sessions = []
for user_id in [2,3,38,41]:
    for prep_id in ['DK55','DK41','DK43']:
        search_dictionary = {'annotation_type':'MARKED_CELL',
                            'FK_annotator_id':user_id,
                            'FK_prep_id':prep_id,
                            'active':1}
        sessions= sessions+controller.query_table(search_dictionary,AnnotationSession)

In [36]:
cell_type_id_to_name

{1: 'Premotor',
 2: 'Starter',
 3: 'Mcherry',
 5: 'Round1',
 7: 'Round3',
 8: 'Samick',
 10: 'round2_multi_threshold',
 11: 'Round3_Sure',
 12: 'Round3_Unsure',
 13: 'connected_cells',
 14: 'false_negative',
 16: 'Round3_Sure_2000',
 17: 'Round3_Unsure_2000',
 18: 'false_negative_2000',
 19: 'connected_cells_2000',
 20: 'Trigeminal Premotor',
 21: 'duplicate_detection',
 22: 'Round4_Sure',
 23: 'Round4_Unsure'}

In [34]:
cell_controller = MarkedCellController()
cell_type_controller = CellTypeController()
cell_type_id_to_name = cell_type_controller.get_cell_type_id_to_name()
annotations = []
time = []
for sessioni in sessions:
    id = sessioni.id
    prep_id = sessioni.FK_prep_id
    annotation = cell_controller.get_marked_cells({'session_id':id})
    if annotation != []:
        if annotation[0].FK_cell_type_id is None:
            continue
        if annotation[0].FK_cell_type_id in [1,16,17,18,19,22,23]:
            time.append(sessioni.created)
            cell_type = cell_type_id_to_name[annotation[0].FK_cell_type_id]
        annotation = get_DataFrame_from_query_result(annotation,prep_id+'_'+cell_type,factor = [0.325,0.325,20])
        annotation.to_csv('/scratch/programming/all_manual_annotations/'+annotation.iloc[0]['name']+'.csv')
        annotations.append(annotation)
time=[i.strftime('%Y-%m-%d %H:%M:%S') for i in time]


In [35]:
master_file = pd.DataFrame({'file_name':[i.iloc[0]['name'] for i in annotations],'created':time,'npoints':[len(i) for i in annotations]})
master_file.to_csv('/scratch/programming/all_manual_annotations/annotation_information.csv')

select * from annotation_session as2 where annotation_type ='MARKED_CELL' and FK_annotator_id in(2,3,36,39) and active =1;

DKXX_cell_type_annotator_source
